In [48]:
# magics: ensures that any changes to the modules loaded below will be re-loaded automatically
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

# load general packages
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
#plt.style.use('seaborn-whitegrid')

# load modules related to this exercise
from educationmodel_3_Lise_1 import EducationModel
import estimate_1 as est 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

In [ ]:
model = EducationModel()

In [ ]:
model.solve()

In [ ]:
sol = model.sol

In [ ]:
sol.d.min()

In [ ]:
par = model.par 
sol= model.sol

In [ ]:
from consav.linear_interp import interp_4d

In [ ]:
# give ability
Dad_educ = family_data.Dad_educ
Mom_educ = family_data.Mom_educ
Family_income = family_data.Family_income
Num_siblings = family_data.Num_siblings
Nuclear = family_data.Nuclear
Urban = family_data.Urban
South = family_data.South


experience = decision_data.exp 
school_time = decision_data.edu


d = decision_data.dummy

util_school  = par.delta0*Dad_educ + par.delta1*Mom_educ + par.delta2*Family_income + par.delta3*Num_siblings + par.delta4*Nuclear + par.delta5*Urban + par.delta6*South
ability_wage = par.gamma0_w*Dad_educ + par.gamma1_w*Mom_educ + par.gamma2_w*Family_income + par.gamma3_w*Num_siblings + par.gamma4_w*Nuclear + par.gamma5_w*Urban + par.gamma6_w*South
ability_employment = par.gamma0_e*Dad_educ + par.gamma1_e*Mom_educ + par.gamma2_e*Family_income + par.gamma3_e*Num_siblings + par.gamma4_e*Nuclear + par.gamma5_e*Urban + par.gamma6_e*South
# ability_wage
# ability_employment

school_time_index =school_time-6

# probability of staying in school given the state
# AND THE ORTHOGONAL TYPE
log_likelihood = 0

epsilon = 1e-10

probabilities = {0: par.p1, 1: par.p2, 2: par.p3, 3: par.p4, 4: par.p5, 5: par.p6}
# Iterate over N and T
for i in range(par.N):
    for t in range(par.T):
        # Iterate through all choices including 0
        for x in range(1, 6):

            experience_i = experience[i + par.N * t]
            for i in range(t):
                #if experience_i equal to nan 
                if np.isnan(experience_i):
                    print('experience_i is nan')
                    experience_i = experience[i + par.N *(t-1)]
            
            if np.isnan(experience_i):
                print('experience_i is still nan')
                experience_i = 0

            sol_x = sol.d[t, x, :, :, :, school_time_index[i + par.N * t], :]
            lik_pr_x = interp_4d(
                par.nuw_fix_grid,
                par.nue_fix_grid,
                par.util_sch_fix_grid,
                par.experience_grid,
                sol_x,
                ability_wage[i],
                ability_employment[i],
                util_school[i],
                experience_i
            )
            clamped_lik_pr_x = np.clip(lik_pr_x, epsilon, 1-epsilon)

            if np.isnan(clamped_lik_pr_x).any() or np.isinf(clamped_lik_pr_x).any():
                print(f'NaN or Inf encountered at i={i}, t={t}, x={x}')
                print(f'clamped_lik_pr_x: {clamped_lik_pr_x}')
                print(f'choice_prob_x: {choice_prob_x}')
                clamped_lik_pr_x = 0.1
                continue

            choice_prob_x = clamped_lik_pr_x * d[i + par.N * t] + (1 - clamped_lik_pr_x) * (1 - d[i + par.N * t])
            #if choice_prob_x < 0:
            #    print('choice_prob_x <= 0, i, t:', i, t)
            #    print('choice_prob_x:', choice_prob_x)
            #    log_likelihood += -np.inf
            #else:

            if np.isnan(choice_prob_x).any() or np.isinf(choice_prob_x).any():
                print(f'NaN or Inf encountered at i={i}, t={t}, x={x}')
                print(f'clamped_lik_pr_x: {clamped_lik_pr_x}')
                print(f'choice_prob_x: {choice_prob_x}')
                continue

            if x in probabilities:
                log_likelihood += probabilities[x] * np.log(choice_prob_x)
            #print('log_likelihood =', log_likelihood)

print('log_likelihood =', log_likelihood)


In [ ]:
sol.d.mean()

In [ ]:
sol.d[0,5,14,:]

In [ ]:
sol.d

## Estimate

In [ ]:
par = model.par

In [ ]:
print(f'Max util_school: {util_school.max()}, Min util_school: {util_school.min()}')
print(f'Max ability_wage: {ability_wage.max()}, Min ability_wage: {ability_wage.min()}')
#print(f'Max ability_employment: {ability_employment.max()}, Min ability_employment: {ability_employment.min()}')


In [49]:
estimate_ = est.estimate_class()
family_data, decision_data = estimate_.read_data()

In [ ]:
decision_data

In [ ]:
delta7= np.linspace(-5,5,5)
log_lik = np.nan + np.zeros(5)

In [ ]:
# Problemet er her, at vi rammer noget negativt. 
for i in range(5):
    pnames = ['delta7']
    theta = np.array([delta7[i]])
    log_lik[i]=estimate_.ll(theta,model, family_data, decision_data, pnames)

In [ ]:
decision_data[(decision_data['id'] == 8)]

In [ ]:
decision_data_interrupt = decision_data[decision_data['dummy_interrup'] == 1]
decision_data_interrupt

In [50]:
filtered_decision_data = decision_data[(decision_data['dummy_school'] == 1)
                                        & (decision_data['dummy_work'] == 1)]

In [51]:
filtered_decision_data

,id,year,dummy_school,edu_going_in,dummy_interrup,dummy_work,exp_going_in,wage
5,6,1980,1,11,0,1,0.00,6.74
1273,1274,1980,1,10,0,1,0.00,9.68
1327,1328,1980,1,10,0,1,0.00,NaN
1583,1584,1980,1,12,0,1,0.00,5.55
1675,1676,1980,1,15,0,1,0.00,5.87
...,...,...,...,...,...,...,...,...
16366,977,1989,1,12,0,1,4.60,7.96
16471,1082,1989,1,12,0,1,7.81,10.75
16526,1137,1989,1,12,0,1,7.00,8.70
16541,1152,1989,1,16,0,1,0.00,7.17


In [ ]:
decision_data_8 = decision_data[decision_data['id'] == 8]

In [ ]:
decision_data_8

In [ ]:
decision_data_1274 = decision_data[decision_data['id'] == 1274]
decision_data_1274

In [ ]:
decision_data_1028 = decision_data[decision_data['id'] == 1028]
decision_data_1028

In [ ]:
filtered_decision_data = decision_data[(decision_data['dummy_work'] == 1)
                                        & (decision_data['dummy_school'] == 1)]

In [ ]:
filtered_decision_data

In [ ]:
dummy_wo

In [ ]:
delta7

In [ ]:
log_lik

In [ ]:
log_lik

In [ ]:
pnames = ['delta7']

In [ ]:
model = est.updatepar(model,pnames,theta)

In [ ]:
def updatepar(par,parnames, parvals):
    """ Update parameters """
    for i,parname in enumerate(parnames):
        parval = parvals[i]
        setattr(par,parname,parval)
    return par

In [ ]:
par = model.par

In [ ]:
updatepar(model.par, pnames, [7])

In [ ]:
par.delta7

In [ ]:
model.par.delta7

In [ ]:
fig = plt.figure(figsize=(6,4),dpi=100)

ax = fig.add_subplot(1,1,1)
ax.set_title('Log likelihood')

ax.plot(delta7,log_lik)

ax.set_xlabel('delta7')
ax.set_ylabel('log likelihood')


fig.tight_layout()

In [ ]:
pnames = ['delta7']
bounds = [(None,None)]
theta0 = [0]

In [ ]:
estimate_.estimate(model, family_data, decision_data, pnames, theta0, bounds)

## Simulate

In [ ]:
model.simulate()

In [ ]:
sim=model.sim

In [ ]:
# Step 1: Check Inputs
print("school_time:", school_time)
print("experience:", experience)
assert not np.isnan(school_time).any(), "school_time contains NaN values"
assert not np.isnan(experience).any(), "experience contains NaN values"

# Step 2: Inspect Initialization and Updates
print("Initialization:")
print("Initial school_time:", sim.school_time_init[i])
print("Initial experience:", sim.experience_init[i])

# Inside the simulation loop
print("Updating school_time and experience:")
print("Updated school_time:", sim.school_time[i, t])
print("Updated experience:", sim.experience[i, t])

# Step 3: Debug wage Calculation
wage_value = self.wage(school_time_index, int(sim.experience[i, t]))
print("Wage value:", wage_value)

# Step 4: Review logwage and logestar Functions
# Check for NaN values in logwage and logestar calculations
logwage_value = self.logwage(school_time_index, int(sim.experience[i, t]))
logestar_value = self.logestar(school_time_index, int(sim.experience[i, t]))
print("logwage:", logwage_value)
print("logestar:", logestar_value)


In [ ]:
mean_d_vectors =[]
mean_school_time_vectors = []
mean_wage_vectors = []
mean_experience_vectors = []

In [ ]:
start = 0 
for i in range(model.par.Nfix):
    end = start + model.par.block_length
    
    # Extract slices
    d_slice = sim.d[start:end]
    school_time_slice = sim.school_time[start:end]
    wage_slice = sim.wage[start:end]
    experience_slice = sim.experience[start:end]

    # Check if slices are empty or contain only NaNs
    if np.isnan(d_slice).all():
        mean_d = np.nan  # or another default value
    else:
        mean_d = np.nanmean(d_slice, axis=0)

    if np.isnan(school_time_slice).all():
        mean_school_time = np.nan  # or another default value
    else:
        mean_school_time = np.nanmean(school_time_slice, axis=0)

    if np.isnan(wage_slice).all():
        mean_wage = np.nan  # or another default value
    else:
        mean_wage = np.nanmean(wage_slice, axis=0)

    if np.isnan(experience_slice).all():
        mean_experience = np.nan  # or another default value
    else:
        mean_experience = np.nanmean(experience_slice, axis=0)

    # Append the computed means to the lists
    mean_d_vectors.append(mean_d)
    mean_school_time_vectors.append(mean_school_time)
    mean_wage_vectors.append(mean_wage)
    mean_experience_vectors.append(mean_experience)

    # Update the start index for the next block
    start = end


In [ ]:
start = 0 
for i in range(model.par.Nfix):
    end = start + model.par.block_length
    mean_d = sim.d[start:end].mean(axis=0)
    mean_school_time = sim.school_time[start:end].mean(axis=0)
    mean_wage = sim.wage[start:end].mean(axis=0, skipna = True)
    mean_d_vectors.append(mean_d)
    mean_school_time_vectors.append(mean_school_time)
    mean_wage_vectors.append(mean_wage)
    mean_experience = sim.experience[start:end].mean(axis=0)
    mean_experience_vectors.append(mean_experience)
    start = end 


In [ ]:
plot_vectors = [mean_d_vectors, mean_school_time_vectors, mean_wage_vectors, mean_experience_vectors]
vector_names = ["Mean schooling choice", "Mean school time", "Mean wage", "Mean experience"]

In [ ]:
for vector, name in zip(plot_vectors, vector_names):
    for i in range(model.par.Nfix):
        plt.plot(vector[i], label=f"{name} for type {i}")
    plt.legend()
    plt.xlabel("Time period")
    plt.ylabel(name)
    plt.title(f"{name} for different types over time")
    plt.show()

## Andet